# Structure Sample

Select 5 plans for each of the following disease sites:
- Breast
- Lung
- GU
- GI 
- Head & Neck



## Imports

In [21]:
import sys
from pathlib import Path
from typing import Dict, List
from random import sample

# Package Imports
import pandas as pd
import xlwings as xw

# Local imports
import varian_query as vq

## SQL Paths and connections

In [22]:
# Relative path to the SQL files
SQL_PATH = Path.cwd()


In [23]:
# Names of servers and databases
DBSERVER = 'ARIADBPV1'
SYSTEMDB = 'VARIAN'
AURA = 'ARIADWPV1'
DWDB = 'variandw'

# Connect to the servers
# This is the primary server for the Varian system
aria_con = vq.connect(DBSERVER, SYSTEMDB)
# This is the production Data Warehouse server
aura_conn = vq.connect(AURA, DWDB)


## Functions

In [24]:
def find_plans(aria_con):
    plan_query = SQL_PATH / 'CompletedPlans_query.sql'
    all_plans = vq.run_query(aria_con, plan_query)
    all_plans.set_index('PlanSetupSer', inplace=True)
    all_plans.drop_duplicates(inplace=True)
    return all_plans


In [25]:
def select_plans(plans_table: pd.DataFrame, site_name: str, num_plans=5) -> pd.DataFrame:
    disease_selection = plans_table.DiseaseSite.str.contains(site_name)
    site_selection = plans_table.SiteRegion.str.contains(site_name)
    selection = disease_selection & site_selection
    plan_nums = list(plans_table.PatientId[selection].index)
    plan_count = len(plan_nums)
    if plan_count < num_plans:
        selected_plans = plan_nums
    else:
        selected_plans = sample(plan_nums, k=num_plans)
    print(f'Selected {len(selected_plans)} of {plan_count} plans for {site_name}')
    return selected_plans


In [26]:
def set_category(dicom_type):
    '''Set the category of the structure based on the DICOM type.
    '''
    category_map = {
        'CTV': 'Target',
        'GTV': 'Target',
        'PTV': 'Target',
        'CAVITY': 'OAR',
        'ORGAN': 'OAR',
        'AVOIDANCE': 'Other',
        'BOLUS': 'Other',
        'CONTRAST_AGENT': 'Other',
        'EXTERNAL': 'Other',
        'IRRAD_VOLUME': 'Other',
        'REGISTRATION': 'Other',
        'TREATED_VOLUME': 'Other',
        'SUPPORT': 'Other',
        'FIXATION': 'Other',
        'DOSE_REGION': 'Other',
        'CONTROL': 'Other',
        'NONE': 'Other',
        }
    if dicom_type in category_map:
        return category_map[dicom_type]
    else:
        return None


### Categorize Structures based On RT ROI Type

| **RT ROI   Type** 	| **Structure Category** 	| **Definition**                                                                                                              	|
|-------------------	|------------------------	|-----------------------------------------------------------------------------------------------------------------------------	|
| CTV               	| Target                 	| Clinical Target Volume (as defined in ICRU 50/62)                                                                           	|
| GTV               	| Target                 	| Gross Tumor Volume (as defined in ICRU 50/62)                                                                               	|
| PTV               	| Target                 	| Planning Target Volume (as defined in   ICRU 50/62)                                                                         	|
| CAVITY            	| OAR                    	| Patient anatomical cavity                                                                                                   	|
| ORGAN             	| OAR                    	| Patient organ                                                                                                               	|
| AVOIDANCE         	| Other                  	| Region in which dose is minimized                                                                                           	|
| BOLUS             	| Other                  	| Material layered onto the patient to   increase high dose provided by external beam therapy to the patient’s skin   surface 	|
| CONTRAST_AGENT    	| Other                  	| Volume into which a contrast agent has been injected                                                                        	|
| EXTERNAL          	| Other                  	| External patient contour                                                                                                    	|
| IRRAD_VOLUME      	| Other                  	| Irradiated Volume (as defined in ICRU 50/62)                                                                                	|
| REGISTRATION      	| Other                  	| Registration ROI                                                                                                            	|
| TREATED_VOLUME    	| Other                  	| Treated volume (as defined in ICRU 50/62)                                                                                   	|
| SUPPORT           	| Other                  	| External patient support device                                                                                             	|
| FIXATION          	| Other                  	| External patient fixation or immobilization device                                                                          	|
| DOSE_REGION       	| Other                  	| ROI to be used as a dose reference                                                                                          	|
| CONTROL ROI       	| Other                  	| to be used in control of dose optimization                                                                                  	|

In [27]:
def select_site(all_plans, site, sample_size=5):
    structures_query = SQL_PATH / 'StructureFromPlan.sql'
    # Select representative plans
    plan_selection = select_plans(all_plans, site, num_plans=sample_size)
    plan_columns = ['PlanSetupId', 'Site Description', 'Diagnosis',
                    'Prescription', 'CreationDate']
    plan_info = all_plans.loc[plan_selection, plan_columns]
    # Get the structures for the selected plans
    structure_sets = []
    for plan_ser in plan_selection:
        structures = vq.run_query(aria_con, structures_query, {'Plan_Ser': plan_ser})
        structure_sets.append(structures)

    all_structures = pd.concat(structure_sets)
    all_structures = all_structures.join(plan_info, on='PlanSetupSer')

    # Remove structures that are not clinical
    non_clinical_prefixes = ('$', 'z', 'Z', 'X', 'x', 'DPV', 'dpv')
    exclusion = all_structures.StructureID.str.startswith(non_clinical_prefixes)
    site_structures = all_structures[~exclusion].copy()
    site_structures['Site'] = site
    # Add a Structure Category for grouping
    # Set structures that do not have a DICOM Type to 'NONE'
    site_structures['DicomType'] = site_structures['DicomType'].fillna('NONE')
    site_structures['Category'] = site_structures.DicomType.apply(set_category)
    selected_columns = ['Site', 'Category', 'DicomType', 'StructureID',
                        'Status', 'GenerationMethod', 'Approval_Date',
                        'PlanSetupId', 'Diagnosis', 'Prescription']
    return site_structures[selected_columns]


In [28]:
# Get info on planes completed since the start of 2024
all_plans = find_plans(aria_con)
structures = select_site(all_plans, site='Head & Neck', sample_size=5)


Selected 5 of 70 plans for Head & Neck


In [29]:
structures

,Site,Category,DicomType,StructureID,Status,GenerationMethod,Approval_Date,PlanSetupId,Diagnosis,Prescription
0,Head & Neck,Other,AVOIDANCE,opt Parotid L,APPROVED,MANUAL,2024-12-10 16:07:58.210,ORAL,1: Malignant neoplasm tongue unspecified ...,6000 cGy in 30 Fractions
1,Head & Neck,Other,AVOIDANCE,PRV BR + op,APPROVED,MANUAL,2024-12-10 16:07:58.210,ORAL,1: Malignant neoplasm tongue unspecified ...,6000 cGy in 30 Fractions
2,Head & Neck,Other,AVOIDANCE,PRV5 BrainStem,APPROVED,MANUAL,2024-12-10 16:07:58.210,ORAL,1: Malignant neoplasm tongue unspecified ...,6000 cGy in 30 Fractions
3,Head & Neck,Other,AVOIDANCE,PRV5 Optics,APPROVED,MANUAL,2024-12-10 16:07:58.210,ORAL,1: Malignant neoplasm tongue unspecified ...,6000 cGy in 30 Fractions
4,Head & Neck,Other,AVOIDANCE,PRV5 SpinalCanal,APPROVED,MANUAL,2024-12-10 16:07:58.210,ORAL,1: Malignant neoplasm tongue unspecified ...,6000 cGy in 30 Fractions
...,...,...,...,...,...,...,...,...,...,...
26,Head & Neck,OAR,ORGAN,Parotid B,APPROVED,MANUAL,2024-05-13 12:19:02.537,OROP,1: Malignant neoplasm tongue unspecified ...,5940 cGy in 54 Fractions
27,Head & Neck,OAR,ORGAN,Parotid L,APPROVED,MANUAL,2024-05-13 12:19:02.537,OROP,1: Malignant neoplasm tongue unspecified ...,5940 cGy in 54 Fractions
28,Head & Neck,OAR,ORGAN,Parotid R,APPROVED,MANUAL,2024-05-13 12:19:02.537,OROP,1: Malignant neoplasm tongue unspecified ...,5940 cGy in 54 Fractions
29,Head & Neck,OAR,ORGAN,SpinalCanal,APPROVED,MANUAL,2024-05-13 12:19:02.537,OROP,1: Malignant neoplasm tongue unspecified ...,5940 cGy in 54 Fractions
